In [1]:
import os
os.getcwd()

'/teamspace/studios/this_studio'

In [2]:
os.chdir("100-class-image-classifaction")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    model_name: str
    model_path: Path
    dataset_path: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [4]:
import os
import tensorflow as tf
from LCIC.constants import *
from LCIC.utils.common import read_yaml, save_json, create_directories
tf.__version__

2024-01-13 09:52:25.719462: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-13 09:52:25.812539: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-13 09:52:25.815618: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-13 09:52:27.482520: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


'2.13.0'

In [8]:
class EvalConfigurationManager:
    def __init__(self, 
                config_filepath= CONFIG_FILE_PATH, 
                params_file_path=PARAMS_FILE_PATH):
                
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)
            

    def get_evaluation_config(self)->EvaluationConfig:

        self.config= self.config.model_eval
        self.params = self.params[self.config.model_name]
        
        eval_cfg= EvaluationConfig(
            model_name= self.config.model_name,
            model_path= self.config.model_path,
            dataset_path= self.config.model_path,
            all_params= self.params,
            mlflow_uri= self.config.mlflow_uri,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )

        return eval_cfg

In [9]:
class EvaluationDataPreprocessing():
    def __init__(self,
                config: EvalConfigurationManager,
                params_path: Path = PARAMS_FILE_PATH):
        self.config = config
        self.params = read_yaml(params_path)
        # setting the params for to have the target size and classes of the model
        self.params = self.params[self.config.model_name]

        self.test_dataset = None

    def __generator(self):
        _test_datagen = ImageDataGenerator(rescale=1./255)
        return _test_datagen

    def get_test_data_set(self):
        test_generator = self.__generator()

        test_set = test_gen.flow_from_directory(
            directory=Path.joinpath(self.config.dataset_path, 'test'),
            target_size=self.params.IMAGE_SIZE[:-1],
            color_mode='rgb',
            class_mode='categorical',
            batch_size=self.config.batch_size,
        )

        self.test_dataset = test_set 
        return self.test_dataset

In [ ]:
class ModelEvaluation:
    def __init__(self, config: EvalConfigurationManager):
        self.config = config
        self.test_data = EvaluationDataPreprocessing(config=self.config).get_test_data_set()
        self.model = None
        self.score = None

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy': self.score[1]}
        save_json(path=Path('reports/score.json'), data= scores)
    
    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self.score = model.evaluate(self.test_data, batch_size=self.config.batch_size, verbose=1)
        self.save_score()

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            print(self.config.all_params)
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({'loss': self.score[0], 'accuracy':self.score[1]})

            # model registry
            if tracking_url_type_store != 'file':
                # register the model
                # There are other ways to use the model registry, which depends on the user case,
                # please refer to the doc for more information:
                # at https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, 'model', registered_model_name=str(self.config.model_name))
            else:
                mlflow.keras.log_model(self.model, "model")

In [ ]:
# Model.evaluate(
#     x=None,
#     y=None,
#     batch_size=None,
#     verbose="auto",
#     sample_weight=None,
#     steps=None,
#     callbacks=None,
#     return_dict=False,
#     **kwargs
# )

In [ ]:
# CallbackList = [
#         ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1,save_weights_only=True, save_best_only=True, mode='max'),
#         ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
#         EarlyStopping(monitor='val_loss', patience=5, mode="min", restore_best_weights=False),
#         CSVLogger(csv_path),
#         TensorBoard(log_dir=TB_log_dir, histogram_freq=1),
#         PlotLossesKeras()
#     ]